# CNN Classification with SUBJ Dataset
<hr>

The __modus operandi__ for text classification is to use __word embedding__ for representing words and a Convolutional neural network to learn how to discriminate documents on classification problems. 

__Yoav Goldberg__ commented in _A Primer on Neural Network Models for Natural Language Processing, 2015._ :
> _The non-linearity of the network, as well as the ability to easily integrate pre-trained
word embeddings, often lead to superior classification accuracy._

He also commented in _Neural Network Methods for Natural Language Processing, 2017_ :
> ... _the CNN is in essence a feature-extracting architecture. ... . The CNNs layer's responsibility is to extract meaningful sub-structures that are useful for the overall prediction task at hand._

We will build a text classification model using CNN model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


sentence  label  split
0     smart and alert , thirteen conversations about...      0  train
1     color , musical bounce and warm seas lapping o...      0  train
2     it is not a mass market entertainment but an u...      0  train
3     a light hearted french film about the spiritua...      0  train
4     my wife is an actress has its moments in looki...      0  train
...                                                 ...    ...    ...
9995  in the end , they discover that balance in lif...      1  train
9996  a counterfeit 1000 tomin bank note is passed i...      1  train
9997  enter the beautiful and mysterious secret agen...      1  train
9998  after listening to a missionary from china spe...      1  train
9999  looking for a short cut to fame , glass concoc...      1  train

[10000 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          5000   5000
1          5000   5000

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN-GRU Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [12]:
def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length, mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [13]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 98, 100)      90100       embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 49, 100)      0           conv1d[0][0]                     
______________________________________________________________________________________________

In [14]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)
                    
                else:
                    print('Done!')

            # evaluate the model
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

Epoch 1/100
180/180 [==============================] - 80s 390ms/step - loss: 0.6719 - accuracy: 0.6254 - val_loss: 0.6167 - val_accuracy: 0.8910
Epoch 2/100
180/180 [==============================] - 63s 351ms/step - loss: 0.5943 - accuracy: 0.9474 - val_loss: 0.5625 - val_accuracy: 0.8810
Epoch 3/100
180/180 [==============================] - 63s 348ms/step - loss: 0.5157 - accuracy: 0.9543 - val_loss: 0.4926 - val_accuracy: 0.9060
Epoch 4/100
180/180 [==============================] - 63s 350ms/step - loss: 0.4412 - accuracy: 0.9758 - val_loss: 0.4472 - val_accuracy: 0.9030
Epoch 5/100
180/180 [==============================] - 63s 349ms/step - loss: 0.3843 - accuracy: 0.9777 - val_loss: 0.4130 - val_accuracy: 0.9040
Epoch 6/100
180/180 [==============================] - 62s 345ms/step - loss: 0.3341 - accuracy: 0.9856 - val_loss: 0.3815 - val_accuracy: 0.9110
Epoch 7/100
180/180 [==============================] - 62s 347ms/step - loss: 0.2969 - accuracy: 0.9872 - val_loss: 0.3730 -

180/180 [==============================] - 88s 487ms/step - loss: 0.1923 - accuracy: 0.9878 - val_loss: 0.2714 - val_accuracy: 0.9270
Epoch 11/100
180/180 [==============================] - 88s 487ms/step - loss: 0.1696 - accuracy: 0.9877 - val_loss: 0.2630 - val_accuracy: 0.9170
Epoch 12/100
180/180 [==============================] - 87s 484ms/step - loss: 0.1487 - accuracy: 0.9871 - val_loss: 0.2502 - val_accuracy: 0.9190
Epoch 13/100
180/180 [==============================] - 87s 484ms/step - loss: 0.1288 - accuracy: 0.9912 - val_loss: 0.2444 - val_accuracy: 0.9210
Epoch 14/100
180/180 [==============================] - 87s 485ms/step - loss: 0.1193 - accuracy: 0.9891 - val_loss: 0.2405 - val_accuracy: 0.9210
Epoch 15/100
180/180 [==============================] - 87s 485ms/step - loss: 0.1062 - accuracy: 0.9909 - val_loss: 0.2398 - val_accuracy: 0.9190
Epoch 16/100
180/180 [==============================] - 86s 478ms/step - loss: 0.0954 - accuracy: 0.9931 - val_loss: 0.2344 - val_a

Epoch 5/100
180/180 [==============================] - 69s 386ms/step - loss: 0.4428 - accuracy: 0.9827 - val_loss: 0.4796 - val_accuracy: 0.8970
Epoch 6/100
180/180 [==============================] - 102s 569ms/step - loss: 0.4131 - accuracy: 0.9900 - val_loss: 0.4630 - val_accuracy: 0.8940
Epoch 7/100
180/180 [==============================] - 101s 560ms/step - loss: 0.3886 - accuracy: 0.9876 - val_loss: 0.4498 - val_accuracy: 0.8850
Epoch 8/100
180/180 [==============================] - 100s 557ms/step - loss: 0.3628 - accuracy: 0.9895 - val_loss: 0.4247 - val_accuracy: 0.9030
Epoch 9/100
180/180 [==============================] - 94s 521ms/step - loss: 0.3411 - accuracy: 0.9889 - val_loss: 0.4136 - val_accuracy: 0.8990
Epoch 10/100
180/180 [==============================] - 97s 539ms/step - loss: 0.3179 - accuracy: 0.9920 - val_loss: 0.4028 - val_accuracy: 0.8930
Epoch 11/100
180/180 [==============================] - 99s 547ms/step - loss: 0.2970 - accuracy: 0.9940 - val_loss: 0.3

Test Accuracy: 90.49999713897705
Done!
Test Accuracy: 90.49999713897705
Epoch 1/100
180/180 [==============================] - 141s 728ms/step - loss: 0.6642 - accuracy: 0.6437 - val_loss: 0.5347 - val_accuracy: 0.8940
Epoch 2/100
180/180 [==============================] - 100s 553ms/step - loss: 0.5138 - accuracy: 0.9374 - val_loss: 0.4752 - val_accuracy: 0.9020
Epoch 3/100
180/180 [==============================] - 101s 563ms/step - loss: 0.4357 - accuracy: 0.9750 - val_loss: 0.4214 - val_accuracy: 0.9090
Epoch 4/100
180/180 [==============================] - 102s 568ms/step - loss: 0.3749 - accuracy: 0.9833 - val_loss: 0.3921 - val_accuracy: 0.9050
Epoch 5/100
180/180 [==============================] - 103s 574ms/step - loss: 0.3286 - accuracy: 0.9840 - val_loss: 0.3680 - val_accuracy: 0.9030
Epoch 6/100
180/180 [==============================] - 103s 570ms/step - loss: 0.2872 - accuracy: 0.9899 - val_loss: 0.3430 - val_accuracy: 0.9080
Epoch 7/100
180/180 [=========================

180/180 [==============================] - 79s 439ms/step - loss: 0.3597 - accuracy: 0.9901 - val_loss: 0.4446 - val_accuracy: 0.8730
Epoch 10/100
180/180 [==============================] - 79s 440ms/step - loss: 0.3390 - accuracy: 0.9890 - val_loss: 0.4123 - val_accuracy: 0.8950
Epoch 11/100
180/180 [==============================] - 81s 448ms/step - loss: 0.3161 - accuracy: 0.9924 - val_loss: 0.3970 - val_accuracy: 0.8980
Epoch 12/100
180/180 [==============================] - 80s 447ms/step - loss: 0.2960 - accuracy: 0.9932 - val_loss: 0.3884 - val_accuracy: 0.8980
Epoch 13/100
180/180 [==============================] - 80s 444ms/step - loss: 0.2764 - accuracy: 0.9952 - val_loss: 0.3711 - val_accuracy: 0.9050
Epoch 14/100
180/180 [==============================] - 79s 440ms/step - loss: 0.2595 - accuracy: 0.9956 - val_loss: 0.3628 - val_accuracy: 0.9010
Epoch 15/100
180/180 [==============================] - 79s 437ms/step - loss: 0.2466 - accuracy: 0.9941 - val_loss: 0.3542 - val_a

...

Epoch 9/100
180/180 [==============================] - 65s 362ms/step - loss: 0.2451 - accuracy: 0.9930 - val_loss: 0.3802 - val_accuracy: 0.8850
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 90.79999923706055
Done!
Test Accuracy: 90.79999923706055
Epoch 1/100
180/180 [==============================] - 83s 423ms/step - loss: 0.6650 - accuracy: 0.6355 - val_loss: 0.6004 - val_accuracy: 0.8890
Epoch 2/100
180/180 [==============================] - 78s 431ms/step - loss: 0.5683 - accuracy: 0.9537 - val_loss: 0.5703 - val_accuracy: 0.8720
Epoch 3/100
180/180 [==============================] - 78s 435ms/step - loss: 0.5221 - accuracy: 0.9731 - val_loss: 0.5196 - val_accuracy: 0.9040
Epoch 4/100
180/180 [==============================] - 78s 434ms/step - loss: 0.4630 - accuracy: 0.9794 - val_loss: 0.4642 - val_accuracy: 0.8760
Epoch 5/100
180/180 [==============================] - 80s 444ms/step - loss: 0.3966 - accuracy: 0.9723 - val_loss

Epoch 16/100
180/180 [==============================] - 79s 439ms/step - loss: 0.0754 - accuracy: 0.9942 - val_loss: 0.2439 - val_accuracy: 0.9160
Epoch 17/100
180/180 [==============================] - 79s 438ms/step - loss: 0.0733 - accuracy: 0.9936 - val_loss: 0.2523 - val_accuracy: 0.9120
Epoch 18/100
180/180 [==============================] - 79s 440ms/step - loss: 0.0663 - accuracy: 0.9954 - val_loss: 0.2561 - val_accuracy: 0.9080
Epoch 19/100
180/180 [==============================] - 79s 440ms/step - loss: 0.0596 - accuracy: 0.9960 - val_loss: 0.2716 - val_accuracy: 0.9040
Epoch 20/100
180/180 [==============================] - 79s 441ms/step - loss: 0.0544 - accuracy: 0.9966 - val_loss: 0.2464 - val_accuracy: 0.9170
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Test Accuracy: 91.79999828338623
Done!
Test Accuracy: 91.79999828338623
Epoch 1/100
180/180 [==============================] - 102s 529ms/step - loss: 0.6740 - accuracy: 0.6460 - va

Epoch 31/100
180/180 [==============================] - 76s 423ms/step - loss: 0.0300 - accuracy: 0.9941 - val_loss: 0.2947 - val_accuracy: 0.9000
Epoch 32/100
180/180 [==============================] - 76s 423ms/step - loss: 0.0305 - accuracy: 0.9944 - val_loss: 0.2816 - val_accuracy: 0.9070
Epoch 33/100
180/180 [==============================] - 76s 422ms/step - loss: 0.0334 - accuracy: 0.9930 - val_loss: 0.2941 - val_accuracy: 0.9000
Epoch 34/100
180/180 [==============================] - 76s 423ms/step - loss: 0.0266 - accuracy: 0.9946 - val_loss: 0.2832 - val_accuracy: 0.9000
Restoring model weights from the end of the best epoch.
Epoch 00034: early stopping
Test Accuracy: 92.40000247955322
Done!
Test Accuracy: 92.40000247955322

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  91.799998  91.700000  92.699999  92.400002  91.900003   
1       relu       2  92.100000  91.299999  91.200000  92.299998  89.300001   
2       relu      

180/180 [==============================] - 94s 485ms/step - loss: 0.6572 - accuracy: 0.6236 - val_loss: 0.5563 - val_accuracy: 0.8900
Epoch 2/100
180/180 [==============================] - 93s 516ms/step - loss: 0.5259 - accuracy: 0.9481 - val_loss: 0.5179 - val_accuracy: 0.9030
Epoch 3/100
180/180 [==============================] - 95s 526ms/step - loss: 0.4768 - accuracy: 0.9722 - val_loss: 0.4869 - val_accuracy: 0.9080
Epoch 4/100
180/180 [==============================] - 95s 528ms/step - loss: 0.4342 - accuracy: 0.9836 - val_loss: 0.4753 - val_accuracy: 0.8870
Epoch 5/100
180/180 [==============================] - 96s 534ms/step - loss: 0.4005 - accuracy: 0.9883 - val_loss: 0.4446 - val_accuracy: 0.8990
Epoch 6/100
180/180 [==============================] - 96s 534ms/step - loss: 0.3695 - accuracy: 0.9916 - val_loss: 0.4322 - val_accuracy: 0.8960
Epoch 7/100
180/180 [==============================] - 97s 537ms/step - loss: 0.3442 - accuracy: 0.9934 - val_loss: 0.4114 - val_accurac

Epoch 3/100
180/180 [==============================] - 88s 487ms/step - loss: 0.4582 - accuracy: 0.9739 - val_loss: 0.4618 - val_accuracy: 0.9340
Epoch 4/100
180/180 [==============================] - 88s 487ms/step - loss: 0.4175 - accuracy: 0.9838 - val_loss: 0.4420 - val_accuracy: 0.9190
Epoch 5/100
180/180 [==============================] - 88s 488ms/step - loss: 0.3850 - accuracy: 0.9868 - val_loss: 0.4223 - val_accuracy: 0.9200
Epoch 6/100
180/180 [==============================] - 87s 482ms/step - loss: 0.3564 - accuracy: 0.9894 - val_loss: 0.3971 - val_accuracy: 0.9280
Epoch 7/100
180/180 [==============================] - 87s 482ms/step - loss: 0.3292 - accuracy: 0.9917 - val_loss: 0.3828 - val_accuracy: 0.9230
Epoch 8/100
180/180 [==============================] - 87s 481ms/step - loss: 0.3066 - accuracy: 0.9918 - val_loss: 0.3679 - val_accuracy: 0.9220
Epoch 9/100
180/180 [==============================] - 87s 482ms/step - loss: 0.2865 - accuracy: 0.9926 - val_loss: 0.3755 -

## Summary

In [16]:
record.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  91.799998  91.700000  92.699999  92.400002  91.900003   
5       relu       6  91.600001  91.500002  91.799998  90.799999  91.399997   
3       relu       4  92.400002  92.500001  91.900003  89.700001  90.600002   
4       relu       5  91.799998  92.299998  91.600001  91.000003  90.799999   
2       relu       3  90.899998  92.199999  90.200001  92.199999  91.700000   
1       relu       2  92.100000  91.299999  91.200000  92.299998  89.300001   

        acc6       acc7       acc8       acc9      acc10        AVG  
0  90.899998  91.299999  92.699999  90.499997  90.899998  91.679999  
5  90.600002  92.100000  93.400002  92.000002  91.100001  91.630000  
3  91.500002  90.899998  91.799998  92.199999  90.799999  91.430001  
4  90.499997  91.100001  91.799998  90.799999  92.400002  91.410000  
2  91.100001  90.300000  91.500002  92.000002  91.600001  91.370000  
1  92.199999  91.100001  91.000003  90.499997  92.299998  91.330000

In [17]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        91.679999

In [18]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_SUBJ.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [19]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [21]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [22]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [23]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [24]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [25]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## MLP-CNN-GRU Model

In [26]:
def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [27]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_61"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_123 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_122 (Embedding)       (None, 100, 300)     300000      input_123[0][0]                  
__________________________________________________________________________________________________
conv1d_61 (Conv1D)              (None, 98, 100)      90100       embedding_122[0][0]              
__________________________________________________________________________________________________
max_pooling1d_61 (MaxPooling1D) (None, 49, 100)      0           conv1d_61[0][0]                  
___________________________________________________________________________________________

## Train and Test the Model

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Epoch 1/100
180/180 [==============================] - 66s 333ms/step - loss: 0.6427 - accuracy: 0.6561 - val_loss: 0.5427 - val_accuracy: 0.8880
Epoch 2/100
180/180 [==============================] - 57s 317ms/step - loss: 0.5341 - accuracy: 0.9029 - val_loss: 0.5217 - val_accuracy: 0.8700
Epoch 3/100
180/180 [==============================] - 57s 315ms/step - loss: 0.5007 - accuracy: 0.9091 - val_loss: 0.4821 - val_accuracy: 0.9090
Epoch 4/100
180/180 [==============================] - 56s 314ms/step - loss: 0.4697 - accuracy: 0.9182 - val_loss: 0.4695 - val_accuracy: 0.8900
Epoch 5/100
180/180 [==============================] - 57s 315ms/step - loss: 0.4437 - accuracy: 0.9224 - val_loss: 0.4394 - val_accuracy: 0.8980
Epoch 6/100
180/180 [==============================] - 56s 313ms/step - loss: 0.4134 - accuracy: 0.9345 - val_loss: 0.4169 - val_accuracy: 0.9150
Epoch 7/100
180/180 [==============================] - 56s 309ms/step - loss: 0.3892 - accuracy: 0.9402 - val_loss: 0.4014 -

180/180 [==============================] - 64s 358ms/step - loss: 0.3386 - accuracy: 0.9450 - val_loss: 0.3832 - val_accuracy: 0.8870
Epoch 8/100
180/180 [==============================] - 66s 368ms/step - loss: 0.3272 - accuracy: 0.9398 - val_loss: 0.3690 - val_accuracy: 0.8930
Epoch 9/100
180/180 [==============================] - 66s 368ms/step - loss: 0.3053 - accuracy: 0.9447 - val_loss: 0.3255 - val_accuracy: 0.9050
Epoch 10/100
180/180 [==============================] - 66s 368ms/step - loss: 0.2669 - accuracy: 0.9496 - val_loss: 0.3089 - val_accuracy: 0.9020
Epoch 11/100
180/180 [==============================] - 66s 369ms/step - loss: 0.2326 - accuracy: 0.9562 - val_loss: 0.2992 - val_accuracy: 0.8960
Epoch 12/100
180/180 [==============================] - 66s 368ms/step - loss: 0.2073 - accuracy: 0.9639 - val_loss: 0.2902 - val_accuracy: 0.9050
Epoch 13/100
180/180 [==============================] - 66s 368ms/step - loss: 0.1902 - accuracy: 0.9671 - val_loss: 0.2767 - val_acc

180/180 [==============================] - 88s 487ms/step - loss: 0.4299 - accuracy: 0.9134 - val_loss: 0.3884 - val_accuracy: 0.9170
Epoch 4/100
180/180 [==============================] - 87s 486ms/step - loss: 0.3877 - accuracy: 0.9233 - val_loss: 0.3661 - val_accuracy: 0.9180
Epoch 5/100
180/180 [==============================] - 86s 479ms/step - loss: 0.3604 - accuracy: 0.9208 - val_loss: 0.3339 - val_accuracy: 0.9150
Epoch 6/100
180/180 [==============================] - 89s 492ms/step - loss: 0.3253 - accuracy: 0.9343 - val_loss: 0.3109 - val_accuracy: 0.9240
Epoch 7/100
180/180 [==============================] - 89s 496ms/step - loss: 0.3041 - accuracy: 0.9332 - val_loss: 0.2965 - val_accuracy: 0.9220
Epoch 8/100
180/180 [==============================] - 86s 479ms/step - loss: 0.2820 - accuracy: 0.9399 - val_loss: 0.2835 - val_accuracy: 0.9210
Epoch 9/100
180/180 [==============================] - 83s 460ms/step - loss: 0.2510 - accuracy: 0.9521 - val_loss: 0.2688 - val_accurac

180/180 [==============================] - 56s 312ms/step - loss: 0.0941 - accuracy: 0.9842 - val_loss: 0.2880 - val_accuracy: 0.9130
Epoch 35/100
180/180 [==============================] - 57s 317ms/step - loss: 0.0899 - accuracy: 0.9851 - val_loss: 0.2937 - val_accuracy: 0.9070
Epoch 36/100
180/180 [==============================] - 68s 376ms/step - loss: 0.0821 - accuracy: 0.9881 - val_loss: 0.3021 - val_accuracy: 0.9080
Restoring model weights from the end of the best epoch.
Epoch 00036: early stopping
Test Accuracy: 91.90000295639038
Done!
Epoch 1/100
180/180 [==============================] - 82s 421ms/step - loss: 0.6864 - accuracy: 0.5495 - val_loss: 0.6526 - val_accuracy: 0.8750
Epoch 2/100
180/180 [==============================] - 85s 475ms/step - loss: 0.6406 - accuracy: 0.8887 - val_loss: 0.6129 - val_accuracy: 0.8940
Epoch 3/100
180/180 [==============================] - 86s 478ms/step - loss: 0.5393 - accuracy: 0.8633 - val_loss: 0.4323 - val_accuracy: 0.8920
Epoch 4/100

180/180 [==============================] - 75s 418ms/step - loss: 0.2079 - accuracy: 0.9747 - val_loss: 0.2678 - val_accuracy: 0.9310
Epoch 20/100
180/180 [==============================] - 75s 417ms/step - loss: 0.1998 - accuracy: 0.9737 - val_loss: 0.2689 - val_accuracy: 0.9280
Epoch 21/100
180/180 [==============================] - 75s 418ms/step - loss: 0.1889 - accuracy: 0.9767 - val_loss: 0.2603 - val_accuracy: 0.9290
Epoch 22/100
180/180 [==============================] - 74s 411ms/step - loss: 0.1764 - accuracy: 0.9798 - val_loss: 0.2552 - val_accuracy: 0.9280
Epoch 23/100
180/180 [==============================] - 74s 413ms/step - loss: 0.1701 - accuracy: 0.9797 - val_loss: 0.2589 - val_accuracy: 0.9230
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Test Accuracy: 94.30000185966492
Done!
Epoch 1/100
180/180 [==============================] - 78s 399ms/step - loss: 0.5859 - accuracy: 0.7432 - val_loss: 0.3995 - val_accuracy: 0.8800
Epoch 2/1

...

Epoch 35/100
180/180 [==============================] - 92s 510ms/step - loss: 0.0813 - accuracy: 0.9900 - val_loss: 0.2647 - val_accuracy: 0.9180
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
Test Accuracy: 92.40000247955322
Done!
Epoch 1/100
180/180 [==============================] - 84s 433ms/step - loss: 0.6474 - accuracy: 0.6097 - val_loss: 0.5311 - val_accuracy: 0.8840
Epoch 2/100
180/180 [==============================] - 87s 483ms/step - loss: 0.5288 - accuracy: 0.7964 - val_loss: 0.4703 - val_accuracy: 0.8870
Epoch 3/100
180/180 [==============================] - 86s 479ms/step - loss: 0.4645 - accuracy: 0.8917 - val_loss: 0.4343 - val_accuracy: 0.8820
Epoch 4/100
180/180 [==============================] - 87s 482ms/step - loss: 0.4148 - accuracy: 0.9135 - val_loss: 0.3965 - val_accuracy: 0.8900
Epoch 5/100
180/180 [==============================] - 87s 481ms/step - loss: 0.3749 - accuracy: 0.9184 - val_loss: 0.3654 - val_accuracy: 0.9030


180/180 [==============================] - 111s 618ms/step - loss: 0.4409 - accuracy: 0.9305 - val_loss: 0.4318 - val_accuracy: 0.9140
Epoch 8/100
180/180 [==============================] - 111s 619ms/step - loss: 0.4165 - accuracy: 0.9354 - val_loss: 0.4187 - val_accuracy: 0.9110
Epoch 9/100
180/180 [==============================] - 111s 620ms/step - loss: 0.3922 - accuracy: 0.9432 - val_loss: 0.3958 - val_accuracy: 0.9210
Epoch 10/100
180/180 [==============================] - 112s 623ms/step - loss: 0.3645 - accuracy: 0.9536 - val_loss: 0.3816 - val_accuracy: 0.9180
Epoch 11/100
180/180 [==============================] - 110s 612ms/step - loss: 0.3436 - accuracy: 0.9582 - val_loss: 0.3683 - val_accuracy: 0.9150
Epoch 12/100
180/180 [==============================] - 111s 614ms/step - loss: 0.3245 - accuracy: 0.9622 - val_loss: 0.3600 - val_accuracy: 0.9160
Epoch 13/100
180/180 [==============================] - 104s 580ms/step - loss: 0.3129 - accuracy: 0.9573 - val_loss: 0.3364 - 

180/180 [==============================] - 109s 603ms/step - loss: 0.1503 - accuracy: 0.9753 - val_loss: 0.2369 - val_accuracy: 0.9250
Epoch 17/100
180/180 [==============================] - 113s 630ms/step - loss: 0.1428 - accuracy: 0.9759 - val_loss: 0.2246 - val_accuracy: 0.9340
Epoch 18/100
180/180 [==============================] - 113s 630ms/step - loss: 0.1293 - accuracy: 0.9798 - val_loss: 0.2351 - val_accuracy: 0.9240
Epoch 19/100
180/180 [==============================] - 112s 624ms/step - loss: 0.1215 - accuracy: 0.9796 - val_loss: 0.2226 - val_accuracy: 0.9270
Epoch 20/100
180/180 [==============================] - 112s 623ms/step - loss: 0.1179 - accuracy: 0.9790 - val_loss: 0.2345 - val_accuracy: 0.9140
Epoch 21/100
180/180 [==============================] - 112s 620ms/step - loss: 0.1119 - accuracy: 0.9808 - val_loss: 0.2329 - val_accuracy: 0.9210
Epoch 22/100
180/180 [==============================] - 112s 624ms/step - loss: 0.1075 - accuracy: 0.9803 - val_loss: 0.2154 

180/180 [==============================] - 89s 497ms/step - loss: 0.4379 - accuracy: 0.9204 - val_loss: 0.4276 - val_accuracy: 0.9070
Epoch 6/100
180/180 [==============================] - 90s 498ms/step - loss: 0.4030 - accuracy: 0.9328 - val_loss: 0.4129 - val_accuracy: 0.9060
Epoch 7/100
180/180 [==============================] - 89s 494ms/step - loss: 0.3811 - accuracy: 0.9371 - val_loss: 0.3952 - val_accuracy: 0.9070
Epoch 8/100
180/180 [==============================] - 90s 498ms/step - loss: 0.3556 - accuracy: 0.9464 - val_loss: 0.3769 - val_accuracy: 0.9120
Epoch 9/100
180/180 [==============================] - 89s 494ms/step - loss: 0.3295 - accuracy: 0.9496 - val_loss: 0.3408 - val_accuracy: 0.9100
Epoch 10/100
180/180 [==============================] - 82s 456ms/step - loss: 0.2884 - accuracy: 0.9552 - val_loss: 0.3106 - val_accuracy: 0.9140
Epoch 11/100
180/180 [==============================] - 82s 456ms/step - loss: 0.2575 - accuracy: 0.9572 - val_loss: 0.2879 - val_accur

180/180 [==============================] - 77s 429ms/step - loss: 0.2224 - accuracy: 0.9564 - val_loss: 0.2852 - val_accuracy: 0.9120
Epoch 11/100
180/180 [==============================] - 77s 430ms/step - loss: 0.2024 - accuracy: 0.9608 - val_loss: 0.2774 - val_accuracy: 0.9160
Epoch 12/100
180/180 [==============================] - 77s 430ms/step - loss: 0.1802 - accuracy: 0.9675 - val_loss: 0.2749 - val_accuracy: 0.9100
Epoch 13/100
180/180 [==============================] - 77s 429ms/step - loss: 0.1671 - accuracy: 0.9692 - val_loss: 0.2703 - val_accuracy: 0.9150
Epoch 14/100
180/180 [==============================] - 76s 423ms/step - loss: 0.1534 - accuracy: 0.9724 - val_loss: 0.2668 - val_accuracy: 0.9120
Epoch 15/100
180/180 [==============================] - 77s 430ms/step - loss: 0.1450 - accuracy: 0.9722 - val_loss: 0.2652 - val_accuracy: 0.9180
Epoch 16/100
180/180 [==============================] - 77s 430ms/step - loss: 0.1300 - accuracy: 0.9769 - val_loss: 0.2602 - val_a

## Summary

In [30]:
record2.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  91.700000  94.400001  91.100001  92.299998  93.400002   
1       relu       2  92.500001  93.000001  91.600001  92.299998  93.000001   
5       relu       6  92.500001  92.600000  91.100001  93.400002  92.699999   
3       relu       4  93.400002  92.699999  91.399997  92.299998  93.099999   
2       relu       3  91.799998  92.400002  93.699998  92.400002  92.799997   
4       relu       5  92.799997  93.099999  91.700000  90.799999  90.499997   

        acc6       acc7       acc8       acc9      acc10        AVG  
0  91.900003  92.699999  94.300002  91.900003  93.099999  92.680001  
1  93.500000  91.700000  92.699999  92.600000  93.000001  92.590000  
5  94.099998  92.000002  91.600001  92.400002  92.100000  92.450001  
3  91.500002  93.400002  92.000002  92.000002  91.399997  92.320000  
2  92.199999  91.700000  91.200000  92.000002  92.199999  92.240000  
4  93.400002  91.299999  92.600000  92.400002  91.200000  91.980000

In [31]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        92.680001

In [32]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_SUBJ_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## MLP-CNN-GRU Model

In [35]:
def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [36]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_122"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_245 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_244 (Embedding)       (None, 100, 300)     300000      input_245[0][0]                  
__________________________________________________________________________________________________
conv1d_122 (Conv1D)             (None, 98, 100)      90100       embedding_244[0][0]              
__________________________________________________________________________________________________
max_pooling1d_122 (MaxPooling1D (None, 49, 100)      0           conv1d_122[0][0]                 
__________________________________________________________________________________________

## Train and Test the Model

In [37]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [38]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Epoch 1/100
180/180 [==============================] - 162s 861ms/step - loss: 0.6089 - accuracy: 0.7052 - val_loss: 0.4709 - val_accuracy: 0.9040
Epoch 2/100
180/180 [==============================] - 148s 820ms/step - loss: 0.4358 - accuracy: 0.9416 - val_loss: 0.4302 - val_accuracy: 0.9190
Epoch 3/100
180/180 [==============================] - 150s 833ms/step - loss: 0.3789 - accuracy: 0.9719 - val_loss: 0.4104 - val_accuracy: 0.9110
Epoch 4/100
180/180 [==============================] - 151s 842ms/step - loss: 0.3406 - accuracy: 0.9837 - val_loss: 0.3926 - val_accuracy: 0.9070
Epoch 5/100
180/180 [==============================] - 151s 842ms/step - loss: 0.3117 - accuracy: 0.9877 - val_loss: 0.3744 - val_accuracy: 0.9140
Epoch 6/100
180/180 [==============================] - 151s 841ms/step - loss: 0.2868 - accuracy: 0.9903 - val_loss: 0.3577 - val_accuracy: 0.9160
Epoch 7/100
180/180 [==============================] - 151s 839ms/step - loss: 0.2641 - accuracy: 0.9928 - val_loss: 0

180/180 [==============================] - 112s 621ms/step - loss: 0.0740 - accuracy: 0.9949 - val_loss: 0.2607 - val_accuracy: 0.9070
Epoch 18/100
180/180 [==============================] - 119s 660ms/step - loss: 0.0658 - accuracy: 0.9970 - val_loss: 0.2487 - val_accuracy: 0.9130
Epoch 19/100
180/180 [==============================] - 120s 669ms/step - loss: 0.0623 - accuracy: 0.9970 - val_loss: 0.2375 - val_accuracy: 0.9190
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Test Accuracy: 92.79999732971191
Done!
Epoch 1/100
180/180 [==============================] - 160s 853ms/step - loss: 0.5955 - accuracy: 0.7372 - val_loss: 0.4559 - val_accuracy: 0.9160
Epoch 2/100
180/180 [==============================] - 149s 827ms/step - loss: 0.4240 - accuracy: 0.9508 - val_loss: 0.4243 - val_accuracy: 0.9180
Epoch 3/100
180/180 [==============================] - 150s 833ms/step - loss: 0.3729 - accuracy: 0.9738 - val_loss: 0.4052 - val_accuracy: 0.9100
Epoch

180/180 [==============================] - 109s 603ms/step - loss: 0.2210 - accuracy: 0.9890 - val_loss: 0.3036 - val_accuracy: 0.9150
Epoch 9/100
180/180 [==============================] - 109s 604ms/step - loss: 0.1956 - accuracy: 0.9917 - val_loss: 0.2996 - val_accuracy: 0.9080
Epoch 10/100
180/180 [==============================] - 109s 605ms/step - loss: 0.1800 - accuracy: 0.9915 - val_loss: 0.2895 - val_accuracy: 0.9080
Epoch 11/100
180/180 [==============================] - 109s 605ms/step - loss: 0.1607 - accuracy: 0.9921 - val_loss: 0.2706 - val_accuracy: 0.9200
Epoch 12/100
180/180 [==============================] - 109s 606ms/step - loss: 0.1453 - accuracy: 0.9928 - val_loss: 0.2691 - val_accuracy: 0.9220
Epoch 13/100
180/180 [==============================] - 109s 606ms/step - loss: 0.1320 - accuracy: 0.9937 - val_loss: 0.2628 - val_accuracy: 0.9190
Epoch 14/100
180/180 [==============================] - 109s 605ms/step - loss: 0.1198 - accuracy: 0.9930 - val_loss: 0.2626 -

180/180 [==============================] - 117s 648ms/step - loss: 0.1442 - accuracy: 0.9929 - val_loss: 0.2629 - val_accuracy: 0.9210
Epoch 7/100
180/180 [==============================] - 116s 646ms/step - loss: 0.1246 - accuracy: 0.9956 - val_loss: 0.2596 - val_accuracy: 0.9110
Epoch 8/100
180/180 [==============================] - 116s 647ms/step - loss: 0.1121 - accuracy: 0.9958 - val_loss: 0.2525 - val_accuracy: 0.9150
Epoch 9/100
180/180 [==============================] - 117s 648ms/step - loss: 0.1013 - accuracy: 0.9958 - val_loss: 0.2537 - val_accuracy: 0.9140
Epoch 10/100
180/180 [==============================] - 116s 646ms/step - loss: 0.0936 - accuracy: 0.9951 - val_loss: 0.2541 - val_accuracy: 0.9110
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 92.1999990940094
Done!
Epoch 1/100
180/180 [==============================] - 162s 861ms/step - loss: 0.6137 - accuracy: 0.7609 - val_loss: 0.4553 - val_accuracy: 0.8930
Epoch 2

180/180 [==============================] - 128s 713ms/step - loss: 0.2849 - accuracy: 0.9827 - val_loss: 0.3411 - val_accuracy: 0.9200
Epoch 8/100
180/180 [==============================] - 125s 694ms/step - loss: 0.2577 - accuracy: 0.9843 - val_loss: 0.3138 - val_accuracy: 0.9210
Epoch 9/100
180/180 [==============================] - 122s 680ms/step - loss: 0.2263 - accuracy: 0.9868 - val_loss: 0.3034 - val_accuracy: 0.9210
Epoch 10/100
180/180 [==============================] - 121s 670ms/step - loss: 0.2054 - accuracy: 0.9856 - val_loss: 0.2883 - val_accuracy: 0.9270
Epoch 11/100
180/180 [==============================] - 120s 669ms/step - loss: 0.1843 - accuracy: 0.9872 - val_loss: 0.2815 - val_accuracy: 0.9200
Epoch 12/100
180/180 [==============================] - 120s 669ms/step - loss: 0.1674 - accuracy: 0.9887 - val_loss: 0.2727 - val_accuracy: 0.9180
Epoch 13/100
180/180 [==============================] - 120s 669ms/step - loss: 0.1546 - accuracy: 0.9861 - val_loss: 0.2663 - 

...

180/180 [==============================] - 196s 1s/step - loss: 0.1107 - accuracy: 0.9946 - val_loss: 0.2582 - val_accuracy: 0.9190
Epoch 10/100
180/180 [==============================] - 200s 1s/step - loss: 0.1015 - accuracy: 0.9944 - val_loss: 0.2604 - val_accuracy: 0.9170
Epoch 11/100
180/180 [==============================] - 198s 1s/step - loss: 0.0915 - accuracy: 0.9943 - val_loss: 0.2471 - val_accuracy: 0.9190
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 92.40000247955322
Done!
Epoch 1/100
180/180 [==============================] - 193s 1s/step - loss: 0.6212 - accuracy: 0.6924 - val_loss: 0.4384 - val_accuracy: 0.9150
Epoch 2/100
180/180 [==============================] - 176s 975ms/step - loss: 0.4133 - accuracy: 0.9358 - val_loss: 0.4025 - val_accuracy: 0.8900
Epoch 3/100
180/180 [==============================] - 174s 967ms/step - loss: 0.3476 - accuracy: 0.9583 - val_loss: 0.3591 - val_accuracy: 0.9120
Epoch 4/100
180/1

180/180 [==============================] - 133s 737ms/step - loss: 0.2366 - accuracy: 0.9939 - val_loss: 0.3466 - val_accuracy: 0.9050
Epoch 12/100
180/180 [==============================] - 132s 736ms/step - loss: 0.2202 - accuracy: 0.9959 - val_loss: 0.3261 - val_accuracy: 0.9130
Epoch 13/100
180/180 [==============================] - 131s 730ms/step - loss: 0.2062 - accuracy: 0.9952 - val_loss: 0.3091 - val_accuracy: 0.9250
Epoch 14/100
180/180 [==============================] - 131s 727ms/step - loss: 0.1955 - accuracy: 0.9948 - val_loss: 0.3144 - val_accuracy: 0.9110
Epoch 15/100
180/180 [==============================] - 137s 759ms/step - loss: 0.1825 - accuracy: 0.9955 - val_loss: 0.2956 - val_accuracy: 0.9220
Epoch 16/100
180/180 [==============================] - 139s 773ms/step - loss: 0.1703 - accuracy: 0.9962 - val_loss: 0.2949 - val_accuracy: 0.9160
Epoch 17/100
180/180 [==============================] - 137s 761ms/step - loss: 0.1604 - accuracy: 0.9965 - val_loss: 0.2896 

180/180 [==============================] - 155s 859ms/step - loss: 0.3421 - accuracy: 0.9894 - val_loss: 0.3847 - val_accuracy: 0.9290
Epoch 7/100
180/180 [==============================] - 154s 858ms/step - loss: 0.3189 - accuracy: 0.9904 - val_loss: 0.3722 - val_accuracy: 0.9210
Epoch 8/100
180/180 [==============================] - 154s 858ms/step - loss: 0.2959 - accuracy: 0.9913 - val_loss: 0.3629 - val_accuracy: 0.9190
Epoch 9/100
180/180 [==============================] - 155s 859ms/step - loss: 0.2747 - accuracy: 0.9930 - val_loss: 0.3413 - val_accuracy: 0.9290
Epoch 10/100
180/180 [==============================] - 154s 858ms/step - loss: 0.2564 - accuracy: 0.9934 - val_loss: 0.3304 - val_accuracy: 0.9280
Epoch 11/100
180/180 [==============================] - 155s 858ms/step - loss: 0.2386 - accuracy: 0.9953 - val_loss: 0.3228 - val_accuracy: 0.9230
Epoch 12/100
180/180 [==============================] - 154s 857ms/step - loss: 0.2242 - accuracy: 0.9945 - val_loss: 0.3150 - v

180/180 [==============================] - 193s 1s/step - loss: 0.3841 - accuracy: 0.9639 - val_loss: 0.4047 - val_accuracy: 0.9220
Epoch 5/100
180/180 [==============================] - 193s 1s/step - loss: 0.3475 - accuracy: 0.9878 - val_loss: 0.3933 - val_accuracy: 0.9100
Epoch 6/100
180/180 [==============================] - 193s 1s/step - loss: 0.3180 - accuracy: 0.9904 - val_loss: 0.3778 - val_accuracy: 0.9100
Epoch 7/100
180/180 [==============================] - 193s 1s/step - loss: 0.2923 - accuracy: 0.9929 - val_loss: 0.3683 - val_accuracy: 0.9040
Epoch 8/100
180/180 [==============================] - 193s 1s/step - loss: 0.2741 - accuracy: 0.9909 - val_loss: 0.3430 - val_accuracy: 0.9180
Epoch 9/100
180/180 [==============================] - 192s 1s/step - loss: 0.2525 - accuracy: 0.9936 - val_loss: 0.3329 - val_accuracy: 0.9130
Epoch 10/100
180/180 [==============================] - 196s 1s/step - loss: 0.2352 - accuracy: 0.9946 - val_loss: 0.3282 - val_accuracy: 0.9090
Epo

180/180 [==============================] - 152s 847ms/step - loss: 0.0930 - accuracy: 0.9952 - val_loss: 0.2713 - val_accuracy: 0.8950
Epoch 14/100
180/180 [==============================] - 152s 847ms/step - loss: 0.0879 - accuracy: 0.9943 - val_loss: 0.2650 - val_accuracy: 0.9060
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 91.90000295639038
Done!
Epoch 1/100
180/180 [==============================] - 227s 1s/step - loss: 0.6569 - accuracy: 0.6103 - val_loss: 0.5371 - val_accuracy: 0.8960
Epoch 2/100
180/180 [==============================] - 220s 1s/step - loss: 0.5195 - accuracy: 0.9007 - val_loss: 0.4934 - val_accuracy: 0.9180
Epoch 3/100
180/180 [==============================] - 221s 1s/step - loss: 0.4625 - accuracy: 0.9406 - val_loss: 0.4688 - val_accuracy: 0.9180
Epoch 4/100
180/180 [==============================] - 221s 1s/step - loss: 0.4239 - accuracy: 0.9562 - val_loss: 0.4407 - val_accuracy: 0.9180
Epoch 5/100
180/18

## Summary

In [39]:
record3.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
1       relu       2  91.799998  92.199999  91.000003  92.299998  92.900002   
0       relu       1  93.000001  92.000002  92.100000  92.799997  92.000002   
2       relu       3  91.299999  93.000001  92.100000  93.099999  92.500001   
3       relu       4  91.600001  93.300003  91.900003  93.199998  92.500001   
5       relu       6  93.400002  92.900002  90.300000  93.000001  92.000002   
4       relu       5  92.699999  91.799998  92.900002  93.099999  92.400002   

        acc6       acc7       acc8       acc9      acc10        AVG  
1  92.699999  93.400002  92.000002  94.400001  93.000001  92.570000  
0  92.900002  92.900002  92.199999  92.400002  92.299998  92.460001  
2  93.199998  90.899998  92.500001  93.099999  92.699999  92.440000  
3  92.500001  91.399997  91.600001  92.199999  93.400002  92.360001  
5  92.199999  92.000002  93.000001  91.900003  92.600000  92.330001  
4  91.500002  92.100000  91.700000  92.500001  92.100000  92.280000

In [40]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_SUBJ_3.xlsx', sheet_name='dynamic')